# ComfyUI Colab — Clean "Leštěnka" Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("Mounted at /content/drive")
import os
BASE = "/content/drive/MyDrive/ComfyUI" if os.path.isdir("/content/drive/MyDrive/ComfyUI") else "/content/ComfyUI"
print("ComfyUI base:", BASE)

In [ ]:
!pip install -U pip wheel setuptools

# Numpy/SciPy pins (fixes "numpy.char" errors)
!pip install --no-deps --upgrade "numpy==1.26.4"
!pip install --upgrade "scipy==1.11.4"

# ONNX stack
!pip install --upgrade "onnx==1.16.2" "onnxruntime-gpu==1.18.1"

# OpenCV (prefer headless to reduce conflicts)
!pip install --upgrade "opencv-python-headless==4.10.0.84"

import os, shutil, subprocess, pathlib

CNODES = f"{BASE}/custom_nodes"
aux_dir = os.path.join(CNODES, "comfyui_controlnet_aux")
if os.path.isdir(aux_dir):
    shutil.rmtree(aux_dir)
pathlib.Path(CNODES).mkdir(parents=True, exist_ok=True)
subprocess.check_call(f"cd '{CNODES}' && git clone https://github.com/Fannovel16/comfyui_controlnet_aux.git", shell=True)

!pip install --upgrade "insightface==0.7.3"
import insightface, os
print("InsightFace version:", insightface.__version__)


In [ ]:
import sys, os, importlib
sys.path.insert(0, f"{BASE}/custom_nodes")
mod = importlib.import_module("comfyui_controlnet_aux")

from comfyui_controlnet_aux.node_wrappers import openpose, dwpose

print("ControlNet Aux import OK — OpenPose + DWPose wrappers available.")

from custom_controlnet_aux.openpose.util import body_estimation
from custom_controlnet_aux.dwpose import DwposeDetector

print("Aux warm-up completed.")


In [ ]:
import os, subprocess, time, re, pathlib

# Install cloudflared if missing
if not pathlib.Path('/usr/local/bin/cloudflared').exists():
    subprocess.run('wget -q -O cloudflared.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb', shell=True, check=True)
    subprocess.run('dpkg -i cloudflared.deb', shell=True, check=True)

os.chdir(BASE)
server = subprocess.Popen(['python', 'main.py', '--listen', '0.0.0.0:8188', '--cuda-device', '0'])

# Start cloudflared tunnel
subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:8188', '--logfile', 'cloudflared.log', '--no-autoupdate'])

# Wait for tunnel to initialize and print the public URL
for _ in range(30):
    time.sleep(1)
    if os.path.exists('cloudflared.log'):
        with open('cloudflared.log') as f:
            for line in f:
                if 'trycloudflare.com' in line:
                    url = re.search(r'https://[0-9a-z-]*\.trycloudflare.com', line).group(0)
                    print('Public URL:', url)
                    raise SystemExit
print('Failed to create Cloudflared tunnel')


In [ ]:
import pathlib

paths = [
    f"{BASE}/models/checkpoints",
    f"{BASE}/models/InstantID/checkpoints",
    f"{BASE}/models/InstantID/checkpoints/controlnet",
    f"{BASE}/input",
]

for p in paths:
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

print(
    "Model folders prepared. Place your models as follows:\n"
    f"  SDXL: {BASE}/models/checkpoints/sd_xl_base_1.0.safetensors\n"
    f"  InstantID adapter: {BASE}/models/InstantID/checkpoints/ip-adapter.bin\n"
    f"  InstantID controlnet: {BASE}/models/InstantID/checkpoints/controlnet/instantid_sdxl.safetensors"
)
